# Search Asset Information

In [ ]:
import sys
import json
import re
import time, datetime
import pandas as pd
import requests, urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

from insightvm_authen import InsightvmAuthentication

In [ ]:
HOSTNAME = InsightvmAuthentication().getHostname()
HEADER = InsightvmAuthentication().getRequestHeader()
# HOSTNAME, HEADER

## 1. Find Active Agent Scan

In [ ]:
def getAssetIDfromIP(assetIP=""):
    getAssetSearchURL = HOSTNAME + "api/3/assets/search"
    payloads = {
        "filters": [
            {
                "field": "ip-address",
                "operator": "is",
                "value": str(assetIP)
            }
        ],
        "match": "any"
    }
    print("\nSearch Asset ID by Asset IP")
    print("Hostname : " + getAssetSearchURL)
    r = requests.post(getAssetSearchURL, data=json.dumps(payloads),
                      headers=HEADER, verify=False)
    if r.status_code == 200:
        r = r.json()
        if r["resources"][0]["id"]:
            assetID = str(r["resources"][0]["id"])
            print("Your Asset ID : " + assetID)
            return assetID
        else:
            print("\n[Not Found] Your Asset IP")
            return None
    else:
        print("[Error] Request Connection Error.")

In [ ]:
def getAssetInfofromID(assetID=""):
    getAssetSearchURL = HOSTNAME + "api/3/assets/" + str(assetID)
    print("\nSearch Asset IP by Asset ID")
    print("Hostname : " + getAssetSearchURL)
    r = requests.get(getAssetSearchURL, headers=HEADER, verify=False)
    if r.status_code == 200:
        r = r.json()
        return r
    else:
        print("[Error] Request Connection Error.")

In [ ]:
def findlastAgentScantime(assetHistory=""):
    agentScan = [a for a in assetHistory if a["type"] == "AGENT-IMPORT"]
    if agentScan != []:
        return agentScan[-1]
    return {}

In [ ]:
def craftagentScanTimeData(ip="",agentScanTime={}):
    if agentScanTime != {}:
        return {
            "ip" : ip,
            "scanLastTime" : agentScanTime["date"],
            "type" : agentScanTime["type"],
        }
    else:
        return {
            "ip" : ip,
            "scanLastTime" : "N/A",
            "type" : "N/A",
        }

In [ ]:
IPS = [""]

In [ ]:
allAssetAgentLastTimeInfo = []

for assetIP in IPS:
    print("IP : " + assetIP)
    assetID = getAssetIDfromIP(assetIP=assetIP)
    if assetID:
        assetInfo = getAssetInfofromID(assetID=assetID)
        agentScanTime=findlastAgentScantime(assetHistory=assetInfo["history"])
        allAssetAgentLastTimeInfo.append(craftagentScanTimeData(ip=assetInfo["ip"],agentScanTime=agentScanTime))
    else:
        print("Not found this IP in insightVM")
    
allAssetAgentLastTimeInfo_df = pd.DataFrame(allAssetAgentLastTimeInfo)

In [ ]:
allAssetAgentLastTimeInfo_df